In [128]:
import os
import pandas as pd

data_path = "/mnt/L-HDD/dataset/splitted.csv"
df = pd.read_csv(data_path)

In [129]:
df

,unique_id,id,time,type,x,y,hit_sound,repeat,length,spinner_time,...,path_41,path_42,path_43,path_44,path_45,path_46,path_47,path_48,path_49,path_50
0,6537,1178020-0,130708.000000,circle,234,84,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6886,1178020-0,165468.000000,slider,361,252,0,1,44.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6885,1178020-0,165324.000000,slider,466,373,0,1,44.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6884,1178020-0,165035.000000,circle,366,50,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6883,1178020-0,164963.000000,circle,307,97,2,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28784,13188,2333780-4,62441.000000,circle,320,212,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28785,13187,2333780-4,61879.000000,slider,188,300,0,1,120.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28786,13186,2333780-4,61504.000000,circle,76,344,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28787,13194,2333780-4,64504.000000,circle,416,324,2,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* unique_id -> pass
* 'id' -> pass
* 'time' -> pass
* 'type' -> categorical
* 'x' ->  / 512
* 'y' -> / 385
* 'hit_sound' -> categorical
* 'repeat' -> log
* 'length' -> log
* 'spinner_time' -> log
* 'beatmap_id' -> pass
* 'rms' -> min-max
* 'beat_length' -> log
* 'meter' -> log
* 'slider_velocity' -> log
* 'sample_set' -> categorical
* 'volume' -> / 100
* 'effects' -> fix - categorical
* 'mfccs' -> min-max globally
* 'curve_type', -> categorical
* 'paths' -> min-max globally

In [130]:
# It's nobarline. When nobarline is on, it flips the 4. bit which means adding 8.

def correct_effect_value(x):
    if(x > 8):
        return x != 5746
    return 0 if x == 8 else x

df["effects"] = df["effects"].apply(correct_effect_value)

In [131]:
categorical_columns = ["type", "hit_sound", "sample_set", "effects", "curve_type"]
df = pd.get_dummies(df, columns=categorical_columns, prefix=categorical_columns)

In [132]:
df["x"] = df["x"] / 512
df["y"] = df["y"] / 385
df["volume"] = df["volume"] / 100

In [133]:
df["slider_velocity"] = df["slider_velocity"] * -1

In [134]:
import numpy as np
df['repeat'] = np.log1p(df['repeat'])
df['length'] = np.log1p(df['length'])
df['spinner_time'] = np.log1p(df['spinner_time'])
df['beat_length'] = np.log1p(df['beat_length'])
df['meter'] = np.log1p(df['meter'])
df['slider_velocity'] = np.log1p(df['slider_velocity'])

In [135]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df["rms"] = scaler.fit_transform(df[["rms"]])

In [136]:
import numpy as np

mfcc_columns= [col for col in df.columns  if col.startswith("mfcc")]
global_max = np.abs(df[mfcc_columns].values).max()
df[mfcc_columns] = df[mfcc_columns] / global_max

In [137]:
df = df.fillna(0)

In [101]:
df[[col for col in df.columns if col.startswith("path")]]

,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,path_10,...,path_16,path_17,path_18,path_19,path_20,path_21,path_22,path_23,path_24,path_25
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,355.0,295.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,459.0,319.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28785,248.0,300.0,300.0,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
path_x_columns = [col for col in df.columns if col.startswith("path_") and int(col.split("_")[1]) % 2 != 0]
path_y_columns = [col for col in df.columns if col.startswith("path_") and int(col.split("_")[1]) % 2 == 0]

In [159]:
global_max = np.abs(df[path_x_columns].values).max()
df[path_x_columns] = df[path_x_columns] / global_max

In [160]:
global_max = np.abs(df[path_y_columns].values).max()
df[path_y_columns] = df[path_y_columns] / global_max

In [162]:
df.describe()

,unique_id,time,x,y,repeat,length,spinner_time,beatmap_id,rms,beat_length,...,path_41,path_42,path_43,path_44,path_45,path_46,path_47,path_48,path_49,path_50
count,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.0,2.878900e+04,28789.000000,28789.000000,...,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000,28789.000000
mean,14410.660808,57798.861129,0.320403,0.316213,0.230708,1.366800,0.0,1.954675e+06,0.393515,3.729416,...,0.000047,0.000035,0.000040,0.000025,0.000043,0.000032,0.000043,0.000032,0.000047,0.000023
std,8317.448867,64939.911762,0.319538,0.323476,0.366665,2.086090,0.0,5.388398e+05,0.188185,2.783359,...,0.005055,0.003463,0.004929,0.003027,0.005391,0.003891,0.005391,0.003891,0.005933,0.002817
min,1.000000,13.931973,0.000000,0.000000,0.000000,0.000000,0.0,1.178020e+06,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7208.000000,115.403175,0.000000,0.000000,0.000000,0.000000,0.0,1.178020e+06,0.243368,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,14416.000000,36870.000000,0.265625,0.236364,0.000000,0.000000,0.0,2.323478e+06,0.392477,5.668022,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21613.000000,100420.000000,0.583984,0.584416,0.693147,4.007333,0.0,2.331063e+06,0.532490,5.809373,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,28810.000000,299319.000000,1.000000,1.000000,2.708050,6.825460,0.0,2.333780e+06,1.000000,7.265130,...,0.710801,0.431373,0.710801,0.431373,0.813589,0.529412,0.813589,0.529412,0.905923,0.352941


In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("/mnt/L-HDD/try_dataset/normalized.csv")

/tmp/ipykernel_52489/4084355681.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/mnt/L-HDD/try_dataset/normalized.csv")


In [3]:
df

,id,time,x,y,repeat,length,spinner_time,beatmap_id,beat_length,meter,...,new_combo_0,new_combo_1,difficulty_rating_0,difficulty_rating_1,difficulty_rating_7,difficulty_rating_8,difficulty_rating_9,difficulty_rating_10,difficulty_rating_11,difficulty_rating_12
0,2338303-0,1375.0,0.197266,0.844156,0.0,0.0,0.0,2338303,6.034684,1.609438,...,False,False,False,False,False,False,False,False,False,False
1,2338303-0,1381.587302,0.0,0.0,0.0,0.0,0.0,2338303,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,2338303-0,1393.197279,0.0,0.0,0.0,0.0,0.0,2338303,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,2338303-0,1404.807256,0.0,0.0,0.0,0.0,0.0,2338303,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,2338303-0,1416.417234,0.0,0.0,0.0,0.0,0.0,2338303,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468596,2342033-3,221448.707483,0.0,0.0,0.0,0.0,0.0,2342033,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
468597,2342033-3,221460.31746,0.0,0.0,0.0,0.0,0.0,2342033,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
468598,2342033-3,221471.927438,0.0,0.0,0.0,0.0,0.0,2342033,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
468599,2342033-3,221483.537415,0.0,0.0,0.0,0.0,0.0,2342033,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN


In [5]:
df[df["difficulty_rating_12"]==True]

,id,time,x,y,repeat,length,spinner_time,beatmap_id,beat_length,meter,...,new_combo_0,new_combo_1,difficulty_rating_0,difficulty_rating_1,difficulty_rating_7,difficulty_rating_8,difficulty_rating_9,difficulty_rating_10,difficulty_rating_11,difficulty_rating_12
